In [2]:
import pandas as pd
import re
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth', None)

## Old

In [ ]:
df = pd.read_csv('/content/Preprocessed_1985 - Preprocessed_1985.csv.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/Preprocessed_1985 - Preprocessed_1985.csv.csv'

In [ ]:
# Step 1: Filter out rows with specified conditions
keywords = ['RT', 'route', 'PO', 'P.O.', 'box', 'P O']
pattern = '|'.join(keywords)
filtered_df = df[~df['ADDRESS_1985'].astype(str).str.contains(pattern, case=False)]
filtered_df = filtered_df[['ID_1985', 'ADDRESS_1985', 'zip code_1985']]

# Step 2: Join number and suffix in the address
def join_number_suffix(address):
    if pd.isna(address):
        return address
    address = str(address)  # Convert address to string
    pattern = r'(\d+)\s+(st|nd|rd|th)'
    modified_address = re.sub(pattern, r'\1\2', address, flags=re.IGNORECASE)
    return modified_address

filtered_df['ADDRESS_1985'] = filtered_df['ADDRESS_1985'].apply(join_number_suffix)

# Step 3: Add 'LUBBOCK, TX, UNITED STATES' or 'TX, UNITED STATES' based on conditions
def add_location(row):
    if pd.isna(row['ADDRESS_1985']):
        return row['ADDRESS_1985']
    if 'lubbock' in row['ADDRESS_1985'].lower():
        return row['ADDRESS_1985'] + ', TX, ' + str(row['zip code_1985']) + ', United states'
    else:
        return row['ADDRESS_1985'] + ', LUBBOCK, TX,' + str(row['zip code_1985']) + ', United states'

filtered_df['ADDRESS_1985'] = filtered_df.apply(add_location, axis=1)

In [ ]:
filtered_df = filtered_df.drop('zip code_1985', axis = 1)

In [ ]:
filtered_df['ADDRESS_1985'] = filtered_df['ADDRESS_1985'].str.replace('TX, TX,', 'TX,')

In [ ]:
filtered_df

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
addresses_df = filtered_df

In [ ]:
!pip install geopy
import geocoder
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
from google.colab import files

In [ ]:
# Define a custom User-Agent for geocoder
geocoder_headers = {
    'User-Agent': 'Retry/1.0 (asheerm@umd.com)'
}

def is_property_level(address):
    # Check if the address is a float (missing or non-string value)
    if isinstance(address, float):
        return False
    # Check if the address starts with a number
    return address.strip()[0].isdigit()

def standardize_address(address):
    # Use geocoder with OpenStreetMap and custom User-Agent
    g = geocoder.osm(address, headers=geocoder_headers, timeout=1.5)
    if g.ok:
        return g.address
    return None

# Define batch size
batch_size = 5000  # Adjust batch size as needed

# Process addresses in batches
for i in tqdm(range(0, len(addresses_df), batch_size), desc="Processing batches"):
    batch = addresses_df[i:i+batch_size]
    standardized_addresses = []
    count = 0
    for address in batch['ADDRESS_1985']:
        if is_property_level(address):
            standardized = standardize_address(address)
        else:
            standardized = None
        standardized_addresses.append(standardized)
        # Print the original and standardized address
        print(f"Original: {address}\nStandardized: {standardized}\n")
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} addresses.")
        # Respect the service's rate limit
        time.sleep(1)
    # Add the standardized addresses to the batch DataFrame
    batch['Standardized_Address'] = standardized_addresses
    # Save the batch result to a new CSV file
    batch.to_csv(f'standardized_addresses_1985__batch_{i//batch_size}.csv', index=False)
    files.download(f'standardized_addresses_1985__batch_{i//batch_size}.csv')

In [ ]:
df['Standardized_Address'] = np.nan

In [ ]:
df1 = pd.read_csv('/content/standardized_addresses_1985__batch_0.csv')
df2 = pd.read_csv('/content/standardized_addresses_1985__batch_1.csv')
df3 = pd.read_csv('/content/standardized_addresses_1985__batch_2.csv')
df4 = pd.read_csv('/content/standardized_addresses_1985__batch_3.csv')
df5 = pd.read_csv('/content/standardized_addresses_1985__batch_4.csv')

In [ ]:
def process_and_update(main_df, df):
    # Iterate over the rows of df
    for index, row in df.iterrows():
        address_1985 = row['ADDRESS_1985']
        standardized_address = row['Standardized_Address']

        # Convert address_1985 and standardized_address to strings to avoid errors
        if pd.notnull(address_1985) and pd.notnull(standardized_address):
            address_1985 = str(address_1985)
            standardized_address = str(standardized_address)

            address_parts = address_1985.split(' ')
            if address_parts[0].isdigit():
                # Check if the number is in Standardized_Address
                if address_parts[0] in standardized_address:
                    # Update the main dataframe
                    main_df.loc[main_df['ID_1985'] == row['ID_1985'], 'Standardized_Address'] = standardized_address

    return main_df

In [ ]:
df['Standardized_Address'].isna().sum()

25978

In [ ]:
main_df = process_and_update(df, df1)
df['Standardized_Address'].isna().sum()

21890

In [ ]:
main_df = process_and_update(df, df2)
df['Standardized_Address'].isna().sum()

18157

In [ ]:
main_df = process_and_update(df, df3)
df['Standardized_Address'].isna().sum()

14109

In [ ]:
main_df = process_and_update(df, df4)
df['Standardized_Address'].isna().sum()

9972

In [ ]:
main_df = process_and_update(df, df5)
df['Standardized_Address'].isna().sum()

9201

In [ ]:
df.to_csv('/content/1985_standardized.csv')

In [ ]:
df = pd.read_csv('/content/1985_standardized.csv')

In [ ]:
df['Address_with_Zip'] = df['ADDRESS_1985'] + ', TX, ' + df['zip code_1985'].astype(str) + ', United states'
filtered_df = df.loc[df['Standardized_Address'].isna(), ['ID_1985', 'Address_with_Zip', 'Standardized_Address']]

In [ ]:
filtered_df.shape

(9201, 3)

In [ ]:
def join_number_suffix(address):
    if pd.isna(address):
        return address
    address = str(address)  # Convert address to string
    pattern = r'(\d+)\s+(st|nd|rd|th)'
    modified_address = re.sub(pattern, r'\1\2', address, flags=re.IGNORECASE)
    return modified_address

filtered_df['Address_with_Zip'] = filtered_df['Address_with_Zip'].apply(join_number_suffix)

In [ ]:
keywords = ['RT', 'route', 'PO', 'P.O.', 'box', 'P O']
pattern = '|'.join(keywords)
filtered_df = filtered_df[~filtered_df['Address_with_Zip'].astype(str).str.contains(pattern, case=False)]

In [ ]:
filtered_df = filtered_df.loc[~filtered_df['Address_with_Zip'].isna(), ['ID_1985', 'Address_with_Zip', 'Standardized_Address']]

In [ ]:
filtered_df.to_csv('1985_retry.csv',index=False)

In [ ]:
filtered_df.shape

(4133, 3)

In [ ]:
df = pd.read_csv('/content/1985_retry.csv')

In [ ]:
df.head()

,ID_1985,Address_with_Zip,Standardized_Address
0,1985_00001,"0402 40TH ST, TX, 79404, United states","402 40th St, Lubbock, TX 79404"
1,1985_00003,"0501 35TH ST, TX, 79404, United states","402 40th St, Lubbock, TX 79404"
2,1985_00004,"0507 35TH ST, TX, 79404, United states","507 35th St, Lubbock, TX 79404"
3,1985_00089,"4424 33RD ST, TX, 79410, United states","4424 33rd St, Lubbock, TX 79410"
4,1985_00107,"4414 32ND ST, TX, 79410, United states","4414 32nd St, Lubbock, TX 79410"


In [ ]:
import numpy as np

In [ ]:
def check_digits_existance (address_with_zip, standard_address):
  first = address_with_zip.split(' ')[0]
  if first.isdigit() and first in standard_address:
    return standard_address
  else:
    return np.nan

In [ ]:
df['Standardized_Address'] = df.apply(lambda row: check_digits_existance(row['Address_with_Zip'],row['Standardized_Address']),axis=1)

In [ ]:
df

,ID_1985,Address_with_Zip,Standardized_Address
0,1985_00001,"0402 40TH ST, TX, 79404, United states",NaN
1,1985_00003,"0501 35TH ST, TX, 79404, United states",NaN
2,1985_00004,"0507 35TH ST, TX, 79404, United states",NaN
3,1985_00089,"4424 33RD ST, TX, 79410, United states","4424 33rd St, Lubbock, TX 79410"
4,1985_00107,"4414 32ND ST, TX, 79410, United states","4414 32nd St, Lubbock, TX 79410"
5,1985_00116,"3702 25TH ST, TX, 79410, United states","3702 25th St, Lubbock, TX 79410"
6,1985_00150,"4604 31ST ST, TX, 79410, United states","4604 31st St, Lubbock, TX 79410"
7,1985_00192,"4623 29TH ST, TX, 79410, United states","4623 29th St, Lubbock, TX 79410"
8,1985_00202,"4610 30TH ST, TX, 79410, United states","4610 30th St, Lubbock, TX 79410"
9,1985_00254,"4010 22ND STREET, TX, 79410, United states","4010 22nd St, Lubbock, TX 79407"


In [ ]:
filtered_df = df[df['Standardized_Address'].isna()]

In [ ]:
addresses_df = df[~df['Standardized_Address'].isna()]

In [ ]:
# Define a custom User-Agent for geocoder
geocoder_headers = {
    'User-Agent': 'Retry/1.0 (asheerm@umd.com)'
}

def is_property_level(address):
    # Check if the address is a float (missing or non-string value)
    if isinstance(address, float):
        return False
    # Check if the address starts with a number
    return address.strip()[0].isdigit()

def standardize_address(address):
    # Use geocoder with OpenStreetMap and custom User-Agent
    g = geocoder.osm(address, headers=geocoder_headers, timeout=1.5)
    if g.ok:
        return g.address
    return None

# Define batch size
batch_size = 1500  # Adjust batch size as needed

# Process addresses in batches
for i in tqdm(range(0, len(addresses_df), batch_size), desc="Processing batches"):
    batch = addresses_df[i:i+batch_size]
    standardized_addresses = []
    count = 0
    for address in batch['Address_with_Zip']:
        if is_property_level(address):
            standardized = standardize_address(address)
        else:
            standardized = None
        standardized_addresses.append(standardized)
        # Print the original and standardized address
        print(f"Original: {address}\nStandardized: {standardized}\n")
        count += 1
        if count % 100 == 0:
            print(f"Processed {count} addresses.")
        # Respect the service's rate limit
        time.sleep(1)
    # Add the standardized addresses to the batch DataFrame
    batch['Standardized_Address'] = standardized_addresses
    # Save the batch result to a new CSV file
    batch.to_csv(f'standardized_addresses_1985_retry_batch_{i//batch_size}.csv', index=False)
    files.download(f'standardized_addresses_1985_retry_batch_{i//batch_size}.csv')

Processing batches:   0%|          | 0/2 [00:00<?, ?it/s]

Original: 4424 33RD ST, TX, 79410, United states
Standardized: 4424, 33rd Street, Maxey Park, Lubbock, Lubbock County, Texas, 79410, United States

Original: 4414 32ND ST, TX, 79410, United states
Standardized: 4414, 32nd Street, Bender Terrace, Lubbock, Lubbock County, Texas, 79410, United States

Original: 3702 25TH ST, TX, 79410, United states
Standardized: 3702, 25th Street, Maxey Park, Lubbock, Lubbock County, Texas, 79410, United States

Original: 4604 31ST ST, TX, 79410, United states
Standardized: 4604, 31st Street, Bender Terrace, Lubbock, Lubbock County, Texas, 79410, United States

Original: 4623 29TH ST, TX, 79410, United states
Standardized: 4623, 29th Street, Bender Terrace, Lubbock, Lubbock County, Texas, 79410, United States

Original: 4610 30TH ST, TX, 79410, United states
Standardized: 4610, 30th Street, Bender Terrace, Lubbock, Lubbock County, Texas, 79410, United States

Original: 4010 22ND STREET, TX, 79410, United states
Standardized: 4010, 22nd Street, Lubbock, L

ERROR:geocoder.base:Status code Unknown from https://nominatim.openstreetmap.org/search: ERROR - HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1.5)


Original: 4406 20TH ST, TX, 79410, United states
Standardized: None

Original: 702 E SLATON HWY, TX, 79404, United states
Standardized: None

Original: 702 E SLATON HWY, TX, 79404, United states
Standardized: None

Original: 4020 21ST ST SUITE 4, TX, 79410, United states
Standardized: None



ERROR:geocoder.base:Status code Unknown from https://nominatim.openstreetmap.org/search: ERROR - HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1.5)


...

ERROR:geocoder.base:Status code Unknown from https://nominatim.openstreetmap.org/search: ERROR - HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1.5)


Original: 708 Loop 289 SE., Lubbock, TX, TX, 79404, United states
Standardized: None

Original: 3615 30th St., Lubbock, TX, TX, 79410, United states
Standardized: 3615, 30th Street, Maxey Park, Lubbock, Lubbock County, Texas, 79410, United States

Original: 536 Greenleaf Drive, Richardson, TX, TX, 75080, United states
Standardized: 536, Greenleaf Drive, Richardson, Dallas County, Texas, 75080, United States

Original: 3703 A 30th St., Lubbock, TX, TX, 79410, United states
Standardized: 30th Street, Tech Terrace U.N.I.T., Lubbock, Lubbock County, Texas, 79410, United States

Original: 209 Cabaret, El Paso, TX, TX, 79912, United states
Standardized: 209, Cabaret Drive, El Paso, El Paso County, Texas, 79912, United States

Original: 5206 Brownfield Rd., Lubbock, TX, TX, 79407, United states
Standardized: None

Original: 4510 Marshall Ave., Lubbock, TX, TX, 79416, United states
Standardized: None

Original: 4518 Marshall, Lubbock, TX, TX, 79416, United states
Standardized: None

Original: 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Processing batches: 100%|██████████| 2/2 [1:34:20<00:00, 2830.50s/it]


In [ ]:
filtered_df

,ID_1985,Address_with_Zip,Standardized_Address
0,1985_00001,"0402 40TH ST, TX, 79404, United states",NaN
1,1985_00003,"0501 35TH ST, TX, 79404, United states",NaN
2,1985_00004,"0507 35TH ST, TX, 79404, United states",NaN
13,1985_00283,"1020 loop 289 SE, TX, 79404, United states",NaN
14,1985_00284,"1020 LOOP 289 SE, TX, 79404, United states",NaN
25,1985_00450,"3301 22ND PL, TX, 79410, United states",NaN
26,1985_00451,"3301 22ND PL, TX, 79410, United states",NaN
27,1985_00454,"4409-B 31ST ST, TX, 79410, United states",NaN
29,1985_00481,"2402 AVE H, TX, 79404, United states",NaN
33,1985_00514,"2204 DATE ST, TX, 79404, United states",NaN


In [ ]:
filtered_df.to_csv('1985_RETRY_1.csv')

## Retry - 30/03/2024

In [3]:
df = pd.read_csv('/content/1985_Standardized_ .csv')

In [4]:
df1985 = df[['ID_1985','ADDRESS_1985', 'Standardized_Address_1985', 'zip code_1985']]

In [5]:
empty_s_addresses = df1985[df1985['Standardized_Address_1985'].isna()]

In [6]:
empty_s_addresses.shape

(7768, 4)

In [7]:
substrings = ['rt', 'box', 'route', 'p o', 'can’t read', "can't read"]

# Create a regular expression pattern with word boundaries for each substring
# Use a non-capturing group (?:) to avoid warnings and match the whole words
pattern = r'\b(?:' + '|'.join(substrings) + r')\b'

df1985_tobechanged = empty_s_addresses[~empty_s_addresses['ADDRESS_1985'].str.contains(pattern, case=False, na=False)]

In [8]:
df1985_tobechanged = df1985_tobechanged[~df1985_tobechanged['ADDRESS_1985'].isna()]

In [9]:
df1985_tobechanged.shape

(3003, 4)

In [10]:
import re

def join_number_suffix(address):
    if pd.isna(address):
        return address
    address = str(address)  # Convert address to string
    pattern = r'(\d+)\s+(st|nd|rd|th)'
    modified_address = re.sub(pattern, r'\1\2', address, flags=re.IGNORECASE)
    return modified_address

df1985_tobechanged['ADDRESS_1985'] = df1985_tobechanged['ADDRESS_1985'].apply(join_number_suffix)
#df1985_tobechanged['ADDRESS_1985'] = df1985_tobechanged['ADDRESS_1985'].astype(str) + ' ' + df1985_tobechanged['zip code_1985'].astype(str)

In [ ]:
def filter_df_by_strings(df, column_name, strings_list):
    # Create a pattern that matches any of the strings in the list
    pattern = r'\b(' + '|'.join(strings_list) + r')\b'

    # Use the pattern to filter the DataFrame
    mask = df[column_name].str.contains(pattern, case=False, na=False)
    return df[mask]

# Filter the DataFrame based on strings in the 'address' column
string_lists = ['HWY','Highway', 'road', 'rd']
filtered_df = filter_df_by_strings(df1985_tobechanged, 'ADDRESS_1985', string_lists)
filtered_df

<ipython-input-70-441c04014126>:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df[column_name].str.contains(pattern, case=False, na=False)


,ID_1985,ADDRESS_1985,Standardized_Address_1985,zip code_1985
280,1985_00281,702 E SLATON HWY,NaN,79404
281,1985_00282,702 E SLATON HWY,NaN,79404
526,1985_00527,4321 BROWNFIELD HIGHWAY,NaN,79410
527,1985_00528,4321 BROWNFIELD HWY,NaN,79410
1002,1985_01003,4425 BROWNFIELD HWY,NaN,79410
1398,1985_01399,4411 BROWNFIELD HWY,NaN,79407
1695,1985_01696,66TH S BROWNFIELD HWY,NaN,79414
1994,1985_01995,2834 N CLOVIS RD,NaN,79415
2055,1985_02056,8333 BROWNFIELD HWY,NaN,79407
2291,1985_02292,3000 BLOOMFIELD RD,NaN,87401


In [ ]:
# Checking the addresses of format:
# 1. string that has two words after splitting as a space and these two should be digits and this should be checked in ADDRESS_1985 column
# 2. should be able to check those before proceeding further.
# 3. Once these are checked, They should be modified to first number space second number followed by (without a space) that number's ordinal suffix and then with a space followed by 'street, lubbock, tx {zip code_1985}'

In [ ]:
import pandas as pd
import re
import geocoder

def ordinal_suffix(number):
    if 11 <= number <= 13:
        return 'th'
    else:
        return {1: 'st', 2: 'nd', 3: 'rd'}.get(number % 10, 'th')

def check_and_modify_address(address, zip_code):
    if pd.isna(address) or pd.isna(zip_code):
        return None

    if not isinstance(address, str):
        return None

    parts = address.split()
    if len(parts) != 2:
        return None

    if not parts[0].isdigit() or not parts[1].isdigit():
        return None

    number1 = int(parts[0])
    number2 = int(parts[1])

    suffix1 = ordinal_suffix(number1)
    suffix2 = ordinal_suffix(number2)

    modified_address = f"{number1} {number2}{suffix2} street, lubbock, tx {zip_code}"
    return modified_address

# Define a custom User-Agent for geocoder
geocoder_headers = {
    'User-Agent': 'Retry/1.0 (asheerm@umd.com)'
}

def standardize_address(address):
    if pd.isna(address):
        return 'None'

    # Use geocoder with OpenStreetMap and custom User-Agent
    g = geocoder.osm(address, headers=geocoder_headers, timeout=5)
    if g.ok:
        return g.address
    return 'None'

# Assuming the data is stored in a DataFrame called 'df1985_tobechanged'
# and the zip code is in a column called 'ZIP_CODE_1985'

df1985_tobechanged['MODIFIED_ADDRESS'] = df1985_tobechanged.apply(lambda row: check_and_modify_address(row['ADDRESS_1985'], row['zip code_1985']), axis=1)

# Standardize the modified addresses
df1985_tobechanged['Standardized_Address_1985'] = df1985_tobechanged['MODIFIED_ADDRESS'].apply(standardize_address)

# Get the IDs of the rows with modified standardized addresses
modified_ids = df1985_tobechanged.loc[df1985_tobechanged['Standardized_Address_1985'] != 'None', 'ID_1985']

# Update the 'df' DataFrame with the standardized addresses based on the modified IDs
df.loc[df['ID_1985'].isin(modified_ids), 'Standardized_Address_1985'] = df1985_tobechanged.loc[df1985_tobechanged['ID_1985'].isin(modified_ids), 'Standardized_Address_1985']

In [ ]:
df

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import pandas as pd
import re
import geocoder

def ordinal_suffix(number):
    if 11 <= number <= 13:
        return 'th'
    else:
        return {1: 'st', 2: 'nd', 3: 'rd'}.get(number % 10, 'th')

def check_and_modify_address(address, zip_code):
    if pd.isna(address) or pd.isna(zip_code):
        return None

    if not isinstance(address, str):
        return None

    parts = address.split()
    if len(parts) != 2:
        return None

    if not parts[0].isdigit() or not parts[1].isdigit():
        return None

    number1 = int(parts[0])
    number2 = int(parts[1])

    suffix1 = ordinal_suffix(number1)
    suffix2 = ordinal_suffix(number2)

    modified_address = f"{number1} {number2}{suffix2} street, lubbock, tx {zip_code}"
    return modified_address

# Define a custom User-Agent for geocoder
geocoder_headers = {
    'User-Agent': 'Retry/1.0 (asheerm@umd.com)'
}

def standardize_address(address, tracking_dict):
    if pd.isna(address):
        tracking_dict['NaN'] += 1
        return 'None'

    # Use geocoder with OpenStreetMap and custom User-Agent
    g = geocoder.osm(address, headers=geocoder_headers, timeout=5)
    if g.ok:
        tracking_dict['Success'] += 1
        return g.address
    else:
        tracking_dict['Failure'] += 1
        return 'None'

def check_pattern(address, pattern_dict):
    if pd.isna(address):
        return

    parts = address.split()
    if len(parts) >= 2 and parts[0].isdigit() and parts[1].isdigit():
        pattern_dict['Present'] += 1
    else:
        pattern_dict['Not Present'] += 1

# Assuming the data is stored in a DataFrame called 'df1985_tobechanged'
# and the zip code is in a column called 'ZIP_CODE_1985'

df1985_tobechanged['MODIFIED_ADDRESS'] = df1985_tobechanged.apply(lambda row: check_and_modify_address(row['ADDRESS_1985'], row['zip code_1985']), axis=1)

# Create a tracking dictionary to monitor the geocoding process
tracking_dict = {'Success': 0, 'Failure': 0, 'NaN': 0}

# Standardize the modified addresses and update the tracking dictionary
df1985_tobechanged['ADDRESS_1985'] = df1985_tobechanged['MODIFIED_ADDRESS'].apply(lambda x: standardize_address(x, tracking_dict))

# Create a pattern dictionary to track the presence of the pattern
pattern_dict = {'Present': 0, 'Not Present': 0}

# Check for the pattern in the standardized addresses and update the pattern dictionary
df1985_tobechanged['ADDRESS_1985'].apply(lambda x: check_pattern(x, pattern_dict))

# Get the IDs of the rows with modified standardized addresses
modified_ids = df1985_tobechanged.loc[df1985_tobechanged['Standardized_Address_1985'] != 'None', 'ID_1985']

# Update the 'df' DataFrame with the standardized addresses based on the modified IDs
df.loc[df['ID_1985'].isin(modified_ids), 'Standardized_Address_1985'] = df1985_tobechanged.loc[df1985_tobechanged['ID_1985'].isin(modified_ids), 'Standardized_Address_1985']

# Print the tracking results
print("Geocoding Tracking:")
print(f"Success: {tracking_dict['Success']}")
print(f"Failure: {tracking_dict['Failure']}")
print(f"NaN: {tracking_dict['NaN']}")

# Print the pattern tracking results
print("\nPattern Tracking:")
print(f"Present: {pattern_dict['Present']}")
print(f"Not Present: {pattern_dict['Not Present']}")

Geocoding Tracking:
Success: 0
Failure: 0
NaN: 3002

Pattern Tracking:
Present: 0
Not Present: 3002


In [ ]:
df.to_csv('1985_s.csv', index=False)